This notebook is based largely on the prior works of Patrick Pantel and Dekang Lin (1999). 

The primary purpose here is to classify a given sentence as having prepositional phrase attachment ambiguity. A syntactic parse is required. 

Secondary purpose will be to enumerate the candidate attachment sites for those sentences with ambiguities. 

Once this work is completed, we can use it to analyze sentences with prepositional phrases, both with and without ambiguities, and look for interesting of distinguishing patterns or characteristics. Following Pantel and Lin's approach, we can use the distribution of words across the unambiguous cases to predict the ambiguous cases in an unsupervised manner. 

Pantel and Lin define ambiguous cases of the form (V, N1, P, N2). 

They define unambiguous cases as of the forms (N1, P, N2) and (V, P, N2). 

I'm not sure I agree with this. What about... 

(P, N2, P, N3)? based on above, this is said to be unambiguous. I disagree. 
    sub (V, P, N2, P, N3)? 
    sub (N1, P, N2, P, N3)? 

I carried my golf clubs with new grips on graphite shafts with cavity-backed blades over my shoulder. 

I carried my golf clubs with new grips over my shoulder. 
I carried my golf clubs with new grips from the pro shop. 

I carried my golf bag with the new irons from the pro shop. 
I carried my golf bag across the 15th fairway to the pro shop. 
I carried my golf bag with my friend Jim across the 15th fairway. 
I carried my golf bag to the pro shop with my friend Jim. 

All of form...

N, V, N, P, N, P, N

Bah! Matches first rule. 

N, V, N, P, N, P, N, P, N

I'm calling that ambiguous. 

hence the only unambiguous cases would be of the form... 

(V, P, N2) or maybe
(N, P, N, V)

The man with the horse offered me a job. 

These heuristics must be ranked... first match fires, done. 
(cooler way to say that?)
disjunctive heuristics? 
decision trees? 

(V, N, P, N) => ambiguous (these cats)
(V, P, N) => unambiguous (yea. them cats)
They say...
N, P, N => unambiguous

I say...

(V, P, N, P, N) => ambiguous (I made this!)
I walked with my golf bag to the clubhouse. 
I walked with my golf bag in a pullcart. 

(P, N, P, N, P, N) => ambiguous

^(N, P, N, V) => unambiguous (I made this!)
The man with the beard sells tacos. 

(P, N, P, N) => unambiguous? (variation of N P N - them cats)
(P) => unknown (presumed unambiguous)
=> no P





In [1]:
import re
import os
import nltk

In [2]:
ex_parse = "(S (PP (IN In) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBZ is) (NP (NP (DT a) (JJ single) (NN tree)) (PP (IN with) (NP (JJ orange) (NNS flowers))) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (DT a) (NN church))))) (PP (IN with) (S (NP (DT the) (JJ open) (NN door)) (VP (VBG facing) (RB forward) (RB left)))))) (. .))"

In [3]:
print(ex_parse)

(S (PP (IN In) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBZ is) (NP (NP (DT a) (JJ single) (NN tree)) (PP (IN with) (NP (JJ orange) (NNS flowers))) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (DT a) (NN church))))) (PP (IN with) (S (NP (DT the) (JJ open) (NN door)) (VP (VBG facing) (RB forward) (RB left)))))) (. .))


In [4]:
pat1 = re.compile(r'[VNP]P')

In [5]:
pat1.findall('(S (VP (VBZ is)')

['VP']

In [6]:
ex_parse = "(S (PP (IN In) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBZ is) (NP (NP (DT a) (JJ single) (NN tree)) (PP (IN with) (NP (JJ orange) (NNS flowers))) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (DT a) (NN church))))) (PP (IN with) (S (NP (DT the) (JJ open) (NN door)) (VP (VBG facing) (RB forward) (RB left)))))) (. .))"
print(ex_parse)

(S (PP (IN In) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBZ is) (NP (NP (DT a) (JJ single) (NN tree)) (PP (IN with) (NP (JJ orange) (NNS flowers))) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (DT a) (NN church))))) (PP (IN with) (S (NP (DT the) (JJ open) (NN door)) (VP (VBG facing) (RB forward) (RB left)))))) (. .))


In [7]:
matches = re.findall(pat1,ex_parse)

In [8]:
phrase_pattern = ''.join([m[0] for m in matches])

In [9]:
phrase_pattern

'PNNVNNPNPNNPNPNV'

In [10]:
# NN -> N
new_phrase_pattern = 'N'.join(phrase_pattern.split('NN'))

In [11]:
new_phrase_pattern

'PNVNPNPNPNPNV'

In [12]:
# (V, N, P, N) => ambiguous (these cats) 
match_obj = re.search('VNPN', new_phrase_pattern)


In [13]:
ambiguous = False
if match_obj:
    ambiguous = True

In [14]:
if not ambiguous: 
    match_obj = re.search('VPNPN', new_phrase_pattern)
    if match_obj:
        ambiguous = True

In [15]:
if not ambiguous:
    match_obj = re.search('NPNPN', new_phrase_pattern)
    if match_obj:
        ambiguous = True

In [16]:
no_prep = False
if not ambiguous: 
    match_obj = re.search('P', new_phrase_pattern)
    if not match_obj:
        no_prep = True

In [17]:
# Result: ambiguous. STOP

In [18]:
x1="NPNPNP"
x2="NNZZFF"
m1=re.search('P',x1)
m2=re.search('P',x2)
if not m1:
    print("not m1")
if not m2:
    print("not m2")

not m2


In [19]:
from itertools import groupby

In [20]:
def has_pp_attachment_ambiguity(parse):
    # https://stackoverflow.com/questions/18799036/python-best-way-to-remove-duplicate-character-from-string
    step0 = re.sub(r'SBAR \(IN', 'PP (IN', parse)
    step1 = [m[0] for m in re.findall(r'[VNP]P',step0)]
    step2 = ''.join(step1)
    step3 = ''.join(ch for ch, _ in groupby(step2))
    phrase_pattern = step3
    
    # If no prep phrase, let's not waste time.
    if not re.search(r'P',phrase_pattern):
        if len(phrase_pattern)>0:
            print("No preps: {}".format(phrase_pattern))
            print(parse)
        return False
    
    # (V, N, P, N) => ambiguous (Pantel&Lin) 
    # Mary ate a salad with a fork. 
    # Mary ate a salad with croutons. 
    match_obj = re.search(r'VNPN', phrase_pattern)
    if match_obj:
        return True
    #(V, P, N, P, N) => ambiguous (I made this!) 
    #I walked with my golf bag to the clubhouse. 
    #I walked with my golf bag in a pullcart.
    match_obj = re.search(r'VPNPN', phrase_pattern)
    if match_obj:
        return True
    # (N, P, N, P, N) => ambiguous (I made this!)
    # In at least one image there is a single tree with orange flowers in front of a church with the open door facing forward left.
    # ... [tree with orange] flowers in front of a church ...
    # ... [tree with orange] flowers with five petals on thick branches... 
    # ([N, P, {N, P, N], P, N})
    match_obj = re.search(r'NPNPN', phrase_pattern)
    if match_obj:
        return True
    # (N, V, P, N) => ambiguous (discovered empirically)
    # ... women wearing white bikinis standing next to the water. 
    # (simplification)
    # ... A woman is wearing a bikini cooking on a gas stove. 
    # ... A woman is holding a spatula cooking in a white bikini. 
    if re.search(r'NVNVPN', phrase_pattern):
        return True
    
    # We have now passed all the checks for ambiguity. 
    
    #^(N, P, N, V) => unambiguous (I made this!)
    # The man with the beard sells tacos. 
    match_obj = re.search(r'^NPNV', phrase_pattern)
    if match_obj:
        print("Pattern match (^NPNV=un): {}".format(phrase_pattern))
    else:
        # ...in the image on the left...
        match_obj = re.search(r'PNPN', phrase_pattern)
        if match_obj:
            print("Pattern match (PNPN=un): {}".format(phrase_pattern))
        else:
            print("New pattern: {}".format(phrase_pattern))
    print(parse)
    return False

In [21]:
def get_pp_attachment_ambiguity(parse):
    # https://stackoverflow.com/questions/18799036/python-best-way-to-remove-duplicate-character-from-string
    step0 = re.sub(r'SBAR \(IN', 'PP (IN', parse)
    step1 = [m[0] for m in re.findall(r'[VNP]P',step0)]
    step2 = ''.join(step1)
    step3 = ''.join(ch for ch, _ in groupby(step2))
    phrase_pattern = step3
    
    # If no prep phrase, let's not waste time.
    if not re.search(r'P',phrase_pattern):
        if len(phrase_pattern)>0:
            print("No preps: {}".format(phrase_pattern))
            print(parse)
        return False, 'P'
    
    # (V, N, P, N) => ambiguous (Pantel&Lin) 
    # Mary ate a salad with a fork. 
    # Mary ate a salad with croutons. 
    match_obj = re.search(r'VNPN', phrase_pattern)
    if match_obj:
        return True, 'VNPN'
    #(V, P, N, P, N) => ambiguous (I made this!) 
    #I walked with my golf bag to the clubhouse. 
    #I walked with my golf bag in a pullcart.
    match_obj = re.search(r'VPNPN', phrase_pattern)
    if match_obj:
        return True, 'VPNPN'
    # (N, P, N, P, N) => ambiguous (I made this!)
    # In at least one image there is a single tree with orange flowers in front of a church with the open door facing forward left.
    # ... [tree with orange] flowers in front of a church ...
    # ... [tree with orange] flowers with five petals on thick branches... 
    # ([N, P, {N, P, N], P, N})
    match_obj = re.search(r'NPNPN', phrase_pattern)
    if match_obj:
        return True, 'NPNPN'
    # (N, V, P, N) => ambiguous (discovered empirically)
    # ... women wearing white bikinis standing next to the water. 
    # (simplification)
    # ... A woman is wearing a bikini cooking on a gas stove. 
    # ... A woman is holding a spatula cooking in a white bikini. 
    if re.search(r'NVNVPN', phrase_pattern):
        return True, 'NVNVPN'
    
    # We have now passed all the checks for ambiguity. 
    
    #^(N, P, N, V) => unambiguous (I made this!)
    # The man with the beard sells tacos. 
    match_obj = re.search(r'^NPNV', phrase_pattern)
    if match_obj:
        print("Pattern match (^NPNV=un): {}".format(phrase_pattern))
        print(parse)
        return False, '^NPNV'
    else:
        # ...in the image on the left...
        match_obj = re.search(r'PNPN', phrase_pattern)
        if match_obj:
            print("Pattern match (PNPN=un): {}".format(phrase_pattern))
            print(parse)
            return False, 'PNPN'
        else:
            print("New pattern: {}".format(phrase_pattern))
    print(parse)
    return False, phrase_pattern

In [22]:
parses = []
num_total = 0
num_found = 0
with open('../data/pp_ambiguity_parses.txt') as ppap_in:
    for line in ppap_in:
        num_total += 1
        if has_pp_attachment_ambiguity(line.strip()):
            num_found += 1

New pattern: NVPNVPNVN
(S (NP (DT A) (JJ large) (JJ brown) (NN dog)) (VP (VBZ is) (PP (IN on) (NP (DT the) (NN couch))) (ADVP (RB next) (PP (IN to) (NP (NP (DT a) (JJ blonde) (JJ haired) (NN woman)) (VP (VBG wearing) (NP (NNS pajamas))))))) (. .))
No preps: NVN
(S (NP (PRP it)) (VP (VBZ 's) (NP (JJ blue) (CC and) (NNP red) (NNP box))) (. .))


In [23]:
print(num_found, num_total)

92 94



(P, N, P, N, P, N) => ambiguous

^(N, P, N, V) => unambiguous (I made this!) 

(P, N, P, N) => unambiguous? (variation of N P N - them cats) (P) => unknown (presumed unambiguous) => no P

In [24]:
parses = []
num_total = 0
num_found = 0
with open('../data/all_parses.txt') as ppap_in:
    for line in ppap_in:
        num_total += 1
        if has_pp_attachment_ambiguity(line.strip()):
            num_found += 1

New pattern: NPN
(NP (NP (QP (RB Only) (CD one)) (NN gerbil)) (PP (IN per) (NP (NN picture))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB exactly) (CD three)) (JJ juvenile) (NNS cheetahs))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD six)) (NNS birds))) (. .) (_SP
No preps: NVN
(S (NP (CD One) (NN dog)) (VP (VBZ is) (VP (VBG wearing) (NP (DT a) (NN leash)))) (. .) (_SP
Pattern match (PNPN=un): PNPNVPN
(S (PP (IN In) (NP (NP (DT the) (NN image)) (PP (IN to) (NP (DT the) (NN right))))) (, ,) (NP (DT the) (NN truck)) (VP (VBZ is) (VP (VBN equipped) (PP (IN with) (NP (DT a) (JJ red) (NN plow))))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD three)) (NNS elephants))) (. .) (_SP
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (PDT All) (DT the) (NNS birds)) (PP (IN in) (NP (DT these) (NNS images)))) (VP (VBP are) (NP (NNS adults) (_SP
No preps: NVNV
(S (S (NP (CD 

In [25]:
num_found

557

In [26]:
parses = []
num_total = 0
num_found = 0
ppaa_pats = {}
with open('../data/all_parses.txt') as ppap_in:
    for line in ppap_in:
        if len(line.strip())<8:
            continue
        num_total += 1
        has_ppaa, patstr = get_pp_attachment_ambiguity(line.strip())
        if has_ppaa:
            num_found += 1
        if not patstr in ppaa_pats:
            ppaa_pats[patstr] = 0
        ppaa_pats[patstr] += 1

New pattern: NPN
(NP (NP (QP (RB Only) (CD one)) (NN gerbil)) (PP (IN per) (NP (NN picture))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB exactly) (CD three)) (JJ juvenile) (NNS cheetahs))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD six)) (NNS birds))) (. .) (_SP
No preps: NVN
(S (NP (CD One) (NN dog)) (VP (VBZ is) (VP (VBG wearing) (NP (DT a) (NN leash)))) (. .) (_SP
Pattern match (PNPN=un): PNPNVPN
(S (PP (IN In) (NP (NP (DT the) (NN image)) (PP (IN to) (NP (DT the) (NN right))))) (, ,) (NP (DT the) (NN truck)) (VP (VBZ is) (VP (VBN equipped) (PP (IN with) (NP (DT a) (JJ red) (NN plow))))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD three)) (NNS elephants))) (. .) (_SP
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (PDT All) (DT the) (NNS birds)) (PP (IN in) (NP (DT these) (NNS images)))) (VP (VBP are) (NP (NNS adults) (_SP
No preps: NVNV
(S (S (NP (CD 

No preps: NV
(S (NP (CD One) (NN bottle)) (VP (VBZ is) (ADJP (ADVP (ADVP (IN at) (JJS least)) (RB partly)) (JJ green))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB exactly) (CD two)) (NNS animals))) (. .) (_SP
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (DT The) (NN bikini)) (PP (IN in) (NP (DT the) (JJ left) (NN image)))) (VP (VBZ has) (NP (DT a) (JJ green) (NN bottom) (NN piece))) (. .) (_SP
New pattern: NVPN
(S (NP (QP (IN At) (RBS least) (CD one)) (NN dog)) (VP (VBZ is) (VP (VBG looking) (ADVP (RB directly)) (PP (IN at) (NP (DT the) (NN camera) (_SP
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (QP (IN At) (RBS least) (CD one))) (PP (IN of) (NP (DT the) (NNS dogs)))) (VP (VBZ has) (NP (VBN uncropped) (, ,) (JJ floppy) (NNS ears))) (. .) (_SP
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (CD One)) (PP (IN of) (NP (DT the) (NNS houses)))) (VP (VBZ has) (NP (DT a) (JJ single) (NN chimney))) (. .) (_SP
Pattern match (^NPNV=un): NPNVPNV
(S (NP (NP (DT The) (NN drum) (

In [27]:
num_total

825

In [28]:
num_found

557

In [29]:
ppaa_pats

{'VNPN': 364,
 'NPN': 3,
 'P': 150,
 'PNPN': 6,
 '^NPNV': 51,
 'VPNPN': 105,
 'NPNPN': 31,
 'PNVNVNV': 1,
 'NVNVPN': 57,
 'NVPNVPN': 6,
 'NVPN': 30,
 'PNVNVN': 2,
 'PNVN': 5,
 'PNVNV': 3,
 'VPN': 1,
 'PNVPN': 4,
 'NVPNVPNVN': 1,
 'NVPNVN': 2,
 'NVPNV': 1,
 'PNVPNVPN': 1,
 'PN': 1}

In [30]:
all_parses = []
with open('../data/annotated_dev.parse') as ppap_in:
    for line in ppap_in:
        all_parses.append(line.strip())

In [31]:
ppaa_parses = []
with open('../data/pp_ambiguity.parse') as ppap_in:
    for line in ppap_in:
        ppaa_parses.append(line.strip())

In [32]:
pos_ambiguous_parses = [parse for parse in all_parses if parse in ppaa_parses]

In [33]:
len(pos_ambiguous_parses)

94

In [34]:
annotated_parses = [(parse, parse in ppaa_parses) for parse in all_parses]

In [35]:
my_predictions = [has_pp_attachment_ambiguity(parse) for parse in all_parses]

New pattern: NPN
(NP (NP (QP (RB Only) (CD one)) (NN gerbil)) (PP (IN per) (NP (NN picture))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB exactly) (CD three)) (JJ juvenile) (NNS cheetahs))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD six)) (NNS birds))) (. .))
No preps: NVN
(S (NP (CD One) (NN dog)) (VP (VBZ is) (VP (VBG wearing) (NP (DT a) (NN leash)))) (. .))
Pattern match (PNPN=un): PNPNVPN
(S (PP (IN In) (NP (NP (DT the) (NN image)) (PP (IN to) (NP (DT the) (NN right))))) (, ,) (NP (DT the) (NN truck)) (VP (VBZ is) (VP (VBN equipped) (PP (IN with) (NP (DT a) (JJ red) (NN plow))))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD three)) (NNS elephants))) (. .))
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (PDT All) (DT the) (NNS birds)) (PP (IN in) (NP (DT these) (NNS images)))) (VP (VBP are) (NP (NNS adults))))
No preps: NVNV
(S (S (NP (CD One) (NN dog) (NN sled) (N

(S (NP (DT Each) (NN image)) (VP (VBZ includes) (NP (RB only) (DT a) (JJ single) (NN crab))) (. .))
No preps: NVNVN
(S (NP (DT The) (JJ left) (NN image)) (VP (VBZ shows) (NP (NP (QP (CD one) (CC or) (JJR more)) (NNS beakers)) (VP (VBG containing) (NP (NP (JJ only) (NN water)) (CC and) (NP (DT no) (NN ice)))))) (. .))
Pattern match (^NPNV=un): NPNVPN
(S (NP (NP (CD One)) (PP (IN of) (NP (DT the) (NNS dogs)))) (VP (VBZ is) (VP (VBG wading) (PP (IN through) (NP (NN water))))) (. .))
New pattern: PNVN
(S (PP (IN In) (NP (DT the) (JJ right) (NN image))) (NP (DT the) (NN kick) (NN drum)) (VP (VBZ has) (NP (DT a) (JJ white) (NN skin))) (. .))
No preps: NVNVN
(S (NP (DT An) (NN image)) (VP (VBZ contains) (NP (NP (DT a) (ADJP (JJ black) (CC and) (JJ white)) (NN dog)) (VP (VBG climbing) (NP (DT a) (NN ladder))))) (. .))
No preps: NVNV
(S (S (NP (CD One) (NN dog) (NN team)) (VP (VBZ is) (VP (VBG wearing) (NP (NNS booties))))) (CC and) (S (NP (DT the) (JJ other) (NN dog) (NN team)) (VP (VBZ is) (R

(S (NP (NP (NNS Penguins) (POS ')) (NNS reflections)) (VP (VBP are) (ADJP (JJ visible)) (PP (IN in) (NP (DT the) (NN water)))) (. .))
Pattern match (PNPN=un): PNPNVN
(S (PP (IN In) (NP (DT the) (JJ left) (NN image))) (NP (NP (DT a) (NN human)) (PP (IN with) (NP (DT a) (NN sleeve) (NN tattoo)))) (VP (VBZ is) (VP (VBG touching) (NP (DT a) (NN dog)))) (. .))
New pattern: PNVN
(S (PP (IN In) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (, ,) (NP (PRP we)) (VP (MD can) (VP (VB see) (NP (DT a) (JJ human) (NN hand)))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (IN at) (JJS least) (CD four)) (NN jelly) (NN fish))) (. .))
No preps: NVNV
(S (NP (EX There)) (VP (VBP are) (NP (NP (QP (RB at) (RBS least) (CD three)) (VBN cooked) (NN triangle) (NNS calzones)) (VP (VBD displayed)))) (. .))
New pattern: PNVPNVPN
(S (PP (IN In) (NP (CD one) (NN image))) (, ,) (NP (DT a) (NN mountain) (NN dog)) (VP (VBZ is) (VP (VBG standing) (SBAR (IN with) (S (NP (PRP$ its) (NN body)) (VP (VBD

In [36]:
their_predictions = [p[1] for p in annotated_parses]

In [37]:
import numpy as np
import sklearn


In [38]:
mine = np.array([int(boo) for boo in my_predictions])
theirs = np.array([int(boo) for boo in their_predictions])
false_positives = [all_parses[i] for i in range(len(all_parses)) if mine[i]>theirs[i]]

In [39]:
len(false_positives)

463

In [40]:
fp_stuff = [get_pp_attachment_ambiguity(parse) for parse in false_positives]

In [41]:
len(fp_stuff)

463

In [42]:
from collections import Counter

In [43]:
Counter([fp[1] for fp in fp_stuff])

Counter({'VNPN': 307, 'VPNPN': 73, 'NPNPN': 31, 'NVNVPN': 52})

In [44]:
interesting = [false_positives[i] for i in range(len(false_positives)) if fp_stuff[i][1]=='VNPN']

In [45]:
len(interesting)

307

In [46]:
interesting[141]

'(S (NP (DT A) (NN person)) (VP (VBZ is) (VP (VBG holding) (NP (DT the) (NN crab)) (PP (IN in) (NP (NP (DT the) (NN image)) (PP (IN on) (NP (DT the) (NN left))))))) (. .))'

In [47]:
def search_parses(regex, parses):
    for parse in parses:
        if re.search(regex, parse):
            yield parse

In [48]:
for parse in search_parses(r'bobby', all_parses):
    print(parse)

(S (NP (DT a) (NNP bobby) (NN pin)) (VP (VBZ is) (VP (VBN pinned) (PP (IN on) (NP (DT a) (NN sweater))) (PP (IN with) (NP (NP (DT a) (JJ collared) (NN shirt)) (ADVP (RB underneath)))))))


In [49]:
ppaa_parses[9]

'(S (S (NP (DT Each) (NN image)) (VP (VBZ contains) (NP (CD one) (JJ non) (JJ -) (JJ standing) (NN spaniel)))) (, ,) (CC and) (S (NP (CD one) (NN image)) (VP (VBZ features) (NP (NP (DT a) (NN spaniel)) (PP (IN with) (NP (NP (NN fur)) (PP (IN in) (NP (NP (NNS shades)) (PP (IN of) (NP (NN brown))))))) (PP (PP (IN with) (NP (DT a) (JJ white) (NN chest) (NN stripe))) (, ,) (VP (VBD posed) (S (VP (VBG sitting) (ADJP (JJ upright))))))))) (. .))'

In [50]:
for parse in all_parses:
    if re.search(r'(spaniel.*){2}',parse):
        print(parse)

(S (S (NP (DT Each) (NN image)) (VP (VBZ contains) (NP (CD one) (JJ non) (JJ -) (JJ standing) (NN spaniel)))) (, ,) (CC and) (S (NP (CD one) (NN image)) (VP (VBZ features) (NP (NP (DT a) (NN spaniel)) (PP (IN with) (NP (NP (NN fur)) (PP (IN in) (NP (NP (NNS shades)) (PP (IN of) (NP (NN brown))))))) (PP (PP (IN with) (NP (DT a) (JJ white) (NN chest) (NN stripe))) (, ,) (VP (VBD posed) (S (VP (VBG sitting) (ADJP (JJ upright))))))))) (. .))


In [51]:
new_parses = ['{})'.format(parse) for parse in all_parses]
all_parses = new_parses

In [52]:
import json

In [53]:
sentences = []
with open('../data/nlvr/nlvr2/data/train.json') as train_in:
    for line in train_in:
        example_dict = json.loads(line.strip())
        sentence = example_dict['sentence']
        sentences.append(sentence)

In [54]:
len(sentences)

86373

In [ ]:
with open('../data/train.txt', 'w') as train_out:
    for sent in sentences:
        train_out.write("{}\n".format(sent))

In [ ]:
dsentences = []
with open('../data/nlvr/nlvr2/data/dev.json') as train_in:
    for line in train_in:
        example_dict = json.loads(line.strip())
        sentence = example_dict['sentence']
        dsentences.append(sentence)

In [ ]:
with open('../data/dev.txt', 'w') as train_out:
    for sent in dsentences:
        train_out.write("{}\n".format(sent))

In [55]:
dev_parses = []
num_total = 0
num_found = 0
ppaa_pats_dev = {}
with open('../data/dev.parse') as ppap_in:
    for line in ppap_in:
        if len(line.strip())<8:
            continue
        dev_parses.append(line.strip())
        num_total += 1
        has_ppaa, patstr = get_pp_attachment_ambiguity(line.strip())
        if has_ppaa:
            num_found += 1
        if not patstr in ppaa_pats_dev:
            ppaa_pats_dev[patstr] = 0
        ppaa_pats_dev[patstr] += 1
        if (num_total%250)==0:
            print("Found {}/{} parses with PP ambiguity.".format(num_found,num_total))

New pattern: PNVNVN
(S (PP (IN IN) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBP are) (NP (NP (QP (RB at) (JJS least) (CD four)) (NN bottle) (NNS rows)) (SBAR (WHNP (WDT that)) (S (ADVP (RB together)) (VP (VBP make) (NP (DT a) (VBG walking) (NN path))))))) (. .))
New pattern: PNVNVN
(S (PP (IN IN) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBP are) (NP (NP (QP (RB at) (JJS least) (CD four)) (NN bottle) (NNS rows)) (SBAR (WHNP (WDT that)) (S (ADVP (RB together)) (VP (VBP make) (NP (DT a) (VBG walking) (NN path))))))) (. .))
New pattern: PNVNVN
(S (PP (IN IN) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBP are) (NP (NP (QP (RB at) (JJS least) (CD four)) (NN bottle) (NNS rows)) (SBAR (WHNP (WDT that)) (S (ADVP (RB together)) (VP (VBP make) (NP (DT a) (VBG walking) (NN path))))))) (. .))
New pattern: PNVNVN
(S (PP (IN IN) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBP are) (N

(S (NP (EX There)) (VP (VBZ is) (NP (NP (DT an) (JJ open) (CD three) (NN ring) (NN binder)) (VP (VBG showing) (NP (DT the) (NN silver) (NNS rings)) (ADVP (RB inside))))) (. .))
No preps: NVNVNV
(S (NP (EX There)) (VP (VBZ is) (NP (NP (DT an) (JJ open) (CD three) (NN ring) (NN binder)) (VP (VBG showing) (NP (DT the) (NN silver) (NNS rings)) (ADVP (RB inside))))) (. .))
No preps: NVNVNV
(S (NP (EX There)) (VP (VBZ is) (NP (NP (DT an) (JJ open) (CD three) (NN ring) (NN binder)) (VP (VBG showing) (NP (DT the) (NN silver) (NNS rings)) (ADVP (RB inside))))) (. .))
No preps: NVN
(S (NP (DT The) (JJ right) (NN image)) (VP (VBZ contains) (NP (QP (RB at) (RBS least) (CD five)) (NNS binders))) (. .))
No preps: NVN
(S (NP (DT The) (JJ right) (NN image)) (VP (VBZ contains) (NP (QP (RB at) (RBS least) (CD five)) (NNS binders))) (. .))
No preps: NVN
(S (NP (DT The) (JJ right) (NN image)) (VP (VBZ contains) (NP (QP (RB at) (RBS least) (CD five)) (NNS binders))) (. .))
No preps: NVN
(S (NP (DT The) (JJ

No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (CD four) (JJ blonde) (NNS women))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (CD four) (JJ blonde) (NNS women))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (CD four) (JJ blonde) (NNS women))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (CD four) (JJ blonde) (NNS women))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB at) (RBS least) (CD three)) (NNS animals))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB at) (RBS least) (CD three)) (NNS animals))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB at) (RBS least) (CD three)) (NNS animals))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB at) (RBS least) (CD three)) (NNS animals))) (. .))
No preps: NVN
(S (NP (NP (DT All) (JJ horned) (NNS animals)) (VP (VBN shown))) (VP (VBP are) (VP (VBG facing) (NP (DT the) (JJ same) (NN direction)))) (. .))
No preps: NVN
(S (NP (NP (

In [56]:
num_found

4875

In [57]:
num_total

7005

In [58]:
ppaa_pats_dev

{'VNPN': 3257,
 'PNVNVN': 15,
 '^NPNV': 512,
 'NVPN': 229,
 'VPNPN': 871,
 'NVPNVPN': 35,
 'P': 1122,
 'NVNVPN': 528,
 'NPNPN': 219,
 'PNPN': 63,
 'PNVN': 38,
 'NVPNV': 11,
 'NVPV': 4,
 'PNVPN': 22,
 'PNVPNVPN': 3,
 'PNVNV': 18,
 'PNVPNV': 4,
 'NVPNVN': 12,
 'VPN': 3,
 'NPN': 10,
 'PNVPVPNVPN': 3,
 'PNVNVNV': 7,
 'PNV': 4,
 'NVPNVPNVN': 4,
 'NVPNVNV': 4,
 'NVPVNPVN': 3,
 'NVPNVNVN': 4}

In [59]:
deduped_parses=np.unique(dev_parses)

In [64]:
num_found=0
num_total=0
ppaa_pats_dev = {}
for parse in deduped_parses:
    has_ppaa, patstr = get_pp_attachment_ambiguity(parse)
    if has_ppaa:
        num_found += 1
    num_total += 1
    if not patstr in ppaa_pats_dev:
        ppaa_pats_dev[patstr] = 0
    ppaa_pats_dev[patstr] += 1
    if (num_total%250)==0:
        print("Found {}/{} parses with PP ambiguity.".format(num_found,num_total))

New pattern: NPN
(FRAG (NP (EX There)) (IN at) (RBS most) (CD one) (DT all) (JJ white) (NNP chihuahua) (PP (IN in) (NP (DT both) (NNS images))) (. .))
New pattern: NVPN
(FRAG (NP (PRP it)) (, ,) (S (VP (VBG sitting) (PP (IN by) (NP (DT a) (NN wine) (NN bottle))))) (. .))
New pattern: NPN
(NP (NP (DT A) (VBN spotted) (NN sting) (NN ray)) (PP (IN in) (NP (DT the) (NN ocean))) (. .))
Pattern match (^NPNV=un): NPNVN
(NP (NP (DT The) (JJ colorful) (NN parrot)) (HYPH -) (S (NP (NP (NN type) (NNS birds)) (PP (IN in) (NP (DT the) (ADJP (JJ left) (CC and) (JJ right)) (NNS images)))) (VP (VBP are) (VP (VBG perching) (CC and) (VBG facing) (NP (DT the) (JJ same) (JJ general) (NN direction))))) (. .))
New pattern: NPN
(NP (NP (QP (RB Only) (CD one)) (NN gerbil)) (PP (IN per) (NP (NN picture))) (. .))
No preps: NV
(NP (NP (QP (RB at) (RBS least) (CD two)) (NN lip) (VBZ glosses)) (NP (NP (CD one) (JJ colored) (NN orange)) (CC and) (NP (NP (CD one)) (VP (JJ colored) (S (ADJP (NN black)))))) (. .))
No 

(S (S (NP (CD One) (NN image)) (VP (VBZ is) (PP (IN from) (PP (IN outside) (NP (DT a) (NN town)))))) (CC and) (S (NP (DT the) (JJ other) (NN image)) (VP (VBZ is) (PP (IN from) (PP (IN inside) (NP (DT a) (NN town)))))) (. .))
No preps: NVNVNVN
(S (S (NP (CD One) (NN image)) (VP (VBZ shows) (S (NP (DT a) (JJ single) (NN standing) (NN cheetah)) (VP (VBG eyeing) (NP (DT the) (NN camera)))))) (, ,) (CC and) (S (NP (DT the) (JJ other) (NN image)) (VP (VBZ shows) (NP (JJ multiple) (, ,) (VBG interacting) (NNS cheetahs)))) (. .))
No preps: NVNVNVN
(S (S (NP (CD One) (NN image)) (VP (VBZ shows) (S (NP (JJ multiple) (NNS dogs)) (VP (VBD posed) (S (VP (VBG sitting) (ADJP (JJ upright)))))))) (, ,) (CC and) (S (NP (DT the) (JJ other) (NN image)) (VP (VBZ shows) (NP (DT a) (JJ single) (JJ large) (HYPH -) (NN breed) (NN dog)))) (. .))
No preps: NVNVNVN
(S (S (NP (CD One) (NNP doberman)) (VP (VP (VBZ faces) (ADVP (RB forward))) (CC and) (VP (VBZ wears) (NP (DT a) (NN collar))))) (, ,) (CC and) (S (NP 

In [65]:
num_found

1410

In [66]:
num_total

2010

In [74]:
straight_dev = {}

In [ ]:
straight_dev['NVPN'] = ppaa_pats_dev['NVPN']
straight_dev['']

In [75]:
ambiguous_patterns = {
    'VPNPN',
    'VNPN',
    '^NPNV',
    'NPNPN'
}
    
unambiguous_patterns = {
    'NPN',
    'NVPN',
    'NVPNVPN',
    'NVNVPN',
    'P',
    'NVPNVPNVN',
    'NVPVNPVN',
    'NVPV',
    'PNVNVN',
    'PNVPNVPN',
    'PNVPNVPNVPN',
    'PNVN',
    'PNVPN',
    'PNVPNV',
    'PNPN',
    'PNV',
    'PNVNVNV',
    'PNVNV',
    'VPN',
    'NVPNV',
    'NVPNVNV',
    'NVPNVNVN'
}

In [67]:
ppaa_pats_dev

{'NPN': 3,
 'NVPN': 63,
 'VPNPN': 246,
 '^NPNV': 146,
 'P': 310,
 'VNPN': 944,
 'NVPNVPN': 11,
 'NVNVPN': 157,
 'NVPNVPNVN': 1,
 'NVPVNPVN': 1,
 'NVPNVN': 3,
 'NPNPN': 63,
 'NVPV': 1,
 'PNVNVN': 5,
 'PNVPNVPN': 1,
 'PNVPVPNVPN': 1,
 'PNVN': 11,
 'PNVPN': 8,
 'PNVPNV': 1,
 'PNPN': 19,
 'PNV': 1,
 'PNVNVNV': 2,
 'PNVNV': 6,
 'VPN': 1,
 'NVPNV': 3,
 'NVPNVNV': 1,
 'NVPNVNVN': 1}

In [76]:
num_ambig = 0
for pattern,count in ppaa_pats_dev.items():
    if pattern in ambiguous_patterns:
        num_ambig += count

In [77]:
num_ambig

1399

In [ ]:
with open('../data/train.txt') as train_in:

In [68]:
len(sentences)

86373

In [69]:
sentset = set()
uniq_sents = []
for sent in sentences:
    if sent not in sentset:
        sentset.add(sent)
        uniq_sents.append(sent)

In [70]:
len(uniq_sents)

22840

In [73]:
with open('../data/traindeduped.txt', 'w') as tout:
    for sent in uniq_sents:
        tout.write("{}\n".format(sent))

In [71]:
tps = set()
uniq_parses = []
with open('../data/traindeduped.parse') as train_parse:
    for line in train_parse:
        s = line.strip()
        if s not in tps:
            tps.add(s)
            uniq_parses.append(s)

In [72]:
len(uniq_parses)

25034

In [78]:
uniq_parses[69]

"(S (NP (DT An) (NN image)) (VP (VBZ shows) (NP (NP (DT a) (NN person) (POS 's)) (NN hand)) (PP (RB around) (NP (QP (RB at) (RBS least) (CD one)) (NNP beagle) (NN dog)))) (. .) (_SP"

In [91]:
#node_start := “([A-Z\.]+\$? “
#node_end := “[^ )]*)+"
#delimiter := “ “
#node_end := “)”
sample = "(S (NP (DT An) (NN image)) (VP (VBZ shows) (NP (NP (DT a) (NN person) (POS 's)) (NN hand)) (PP (RB around) (NP (QP (RB at) (RBS least) (CD one)) (NNP beagle) (NN dog)))) (. .)"

def parse_parse(parse):
    node_start_re = re.compile(r'\(([^ ]+)')
    node_end_re = re.compile(r'([^ )]*)(\)+)')

    tags = set()
    words = set()

    tokens = parse.strip().split()
    context_stack = []
    for token in tokens:
        match = re.match(node_start_re, token)
        if match:
            tags.add(match.group(1))
            context_stack.append(match.group(1))
        elif len(context_stack)==0:
            raise ValueError("'{}' is not a valid start token.".format(token))
        else:
            match = re.match(node_end_re, token)
            if match:
                stuff = match.groups()
                words.add(stuff[0])
                context_stack = context_stack[:-(len(stuff[1]))]
            else:
                raise ValueError("Unrecognized token '{}'".format(token))

    return words, tags

words, tags = parse_parse(sample)
print("words", words)
print("tags", tags)
    

words {'dog', 'person', 'a', "'s", 'shows', 'least', 'one', '.', 'image', 'beagle', 'at', 'hand', 'around', 'An'}
tags {'VBZ', 'DT', 'RB', 'POS', 'S', 'QP', 'RBS', 'PP', '.', 'VP', 'CD', 'NN', 'NNP', 'NP'}


In [85]:
tps = set()
uniq_parses = []
with open('../data/devdeduped.parse') as train_parse:
    for line in train_parse:
        s = line.strip()
        if s not in tps:
            tps.add(s)
            uniq_parses.append(s)

In [86]:
len(uniq_parses)

2010

In [95]:
words = set()
tags = set()
ambiguous_parses = {}
unambiguous_parses = {}
for parse in uniq_parses:
    has_ambig, matching_pattern = get_pp_attachment_ambiguity(parse)
    if has_ambig:
        if matching_pattern not in ambiguous_parses:
            ambiguous_parses[matching_pattern]=[]
        ambiguous_parses[matching_pattern].append(parse)
    else:
        if matching_pattern not in unambiguous_parses:
            unambiguous_parses[matching_pattern] = []
        unambiguous_parses[matching_pattern].append(parse)
    new_words, new_tags = parse_parse(parse)
    words.update(new_words)
    tags.update(new_tags)

New pattern: PNVNVN
(S (PP (IN IN) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBP are) (NP (NP (QP (RB at) (JJS least) (CD four)) (NN bottle) (NNS rows)) (SBAR (WHNP (WDT that)) (S (ADVP (RB together)) (VP (VBP make) (NP (DT a) (VBG walking) (NN path))))))) (. .))
Pattern match (^NPNV=un): NPNVPN
(S (NP (NP (DT All)) (PP (IN of) (NP (DT the) (NN beer)))) (VP (VBZ is) (PP (IN in) (NP (NNS bottles)))) (. .))
New pattern: NVPN
(S (NP (QP (RB Only) (CD one)) (NN woman)) (VP (VBZ is) (VP (VBG drinking) (PP (IN in) (NP (DT the) (JJ left) (NN image))))) (. .))
New pattern: NVPNVPN
(S (S (NP (DT A) (NN mitten)) (VP (VBZ is) (VP (VBG being) (VP (VBN worn) (PP (IN in) (NP (CD one) (NN image))))))) (CC and) (S (NP (DT the) (NNS mittens)) (VP (VBP are) (RB not) (VP (VBG being) (VP (VBN worn) (PP (IN in) (NP (DT the) (JJ other) (NN image))))))) (. .))
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (DT The) (NNS screens)) (PP (IN in) (NP (DT the) (JJ left) (CC and) (JJ right

Pattern match (^NPNV=un): NPNVPN
(S (NP (NP (QP (RB At) (RBS least) (CD one))) (PP (IN of) (NP (NNS anemones)))) (VP (VBZ is) (ADJP (JJ white)) (PP (IN with) (NP (JJ dark) (NNS stripes)))) (. .))
New pattern: NVPN
(S (NP (QP (RB At) (RBS least) (CD one)) (NN fish)) (VP (VBZ is) (VP (VBG facing) (VBN left) (PP (IN in) (NP (DT the) (JJ right) (NN image))))) (. .))
New pattern: PNVPN
(S (PP (IN in) (NP (DT the) (JJ right) (NN image))) (NP (DT an) (NN anemone)) (VP (VBZ is) (PP (IN on) (NP (DT a) (NN grate)))))
No preps: NVN
(S (NP (DT The) (JJ left) (CC and) (JJ right) (NN image)) (VP (VBZ contains) (NP (QP (IN at) (RBS least) (CD two)) (JJ blue) (NN snow) (NN truck))) (. .))
New pattern: NVPN
(S (NP (CD One) (NN image)) (VP (VBZ is) (PP (IN from) (PP (IN inside) (NP (DT a) (NN snow) (NN plow))))) (. .))
New pattern: NVPN
(S (NP (CD One) (NN passenger) (NN automobile)) (VP (VBZ is) (VP (VBG driving) (PP (IN in) (NP (DT the) (NN snow))))) (. .))
Pattern match (PNPN=un): PNPNVPN
(S (PP (IN 

In [96]:
tags

{"''",
 ',',
 '-LRB-',
 '-RRB-',
 '.',
 ':',
 'ADJP',
 'ADVP',
 'CC',
 'CD',
 'CONJP',
 'DT',
 'EX',
 'FRAG',
 'FW',
 'HYPH',
 'IN',
 'INTJ',
 'JJ',
 'JJR',
 'JJS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'NP',
 'NX',
 'PDT',
 'POS',
 'PP',
 'PRN',
 'PRP',
 'PRP$',
 'PRT',
 'QP',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'S',
 'SBAR',
 'SINV',
 'SYM',
 'TO',
 'UCP',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'VP',
 'WDT',
 'WHADVP',
 'WHNP',
 'WP',
 'WP$',
 'WRB',
 '_SP',
 '``'}

In [97]:
tot=0
for pat,parses in ambiguous_parses.items():
    print("{}: {}".format(pat, len(parses)))
    tot+=len(parses)

VNPN: 944
VPNPN: 246
NVNVPN: 157
NPNPN: 63


In [98]:
tot

1410

In [103]:
def get_all_ambiguities(parse, pattern=None):
    '''
    Want (pattern, parse, [[PP1, [cand11, cand12, ...]], [PP2, [cand21, cand22, ..., cand2N]]])
    '''
    node_start_re = re.compile(r'\(([^ ]+)')
    node_end_re = re.compile(r'([^ )]*)(\)+)')

    tags = set()
    words = set()
    tagged = []
    
    tokens = parse.strip().split()
    context_stack = []
    phrase_tags = {'VP','NP','PP','WHNP'}
    verb_tags = {'VBZ','VBG','VBP','VBN'}
    noun_tags = {'NN','NNP','NNPS','NNS'}
    last_noun = None
    last_verb = None
    last_prep = None
    heads = []
    
    for token in tokens:
        match = re.match(node_start_re, token)
        if match:
            tag = match.group(1)
            tags.add(tag)
            phrase_context = [tag for tag in context_stack if tag in phrase_tags]
            context_stack.append(tag)
            if tag in phrase_tags:
                head
        elif len(context_stack)==0:
            raise ValueError("'{}' is not a valid start token.".format(token))
        else:
            match = re.match(node_end_re, token)
            if match:
                stuff = match.groups()
                word = stuff[0]
                closing_tags = stuff[1]
                words.add(word)
                popped_context = context_stack[-(len(closing_tags)):]
                pos_tag = popped_context[-1]
                tagged.append((word,pos_tag))
                if pos_tag in noun_tags:
                    last_noun = (word,pos_tag)
                elif pos_tag in verb_tags:
                    last_verb = (word,pos_tag)
                elif pos_tag == 'IN':
                    last_prep = (word,pos_tag)
                popped_context = popped_context[:-1]
                for tag in popped_context:
                    if tag not in phrase_tags:
                        continue
                    if tag == 'PP' and last_prep is not None:
                        heads.append((last_prep,tag))
                        last_prep = None
                    elif tag == 'VP' and last_verb is not None:
                        heads.append((last_verb,tag))
                        last_verb = None
                    elif tag == 'NP' and last_noun is not None:
                        heads.append((last_noun,tag))
                        last_noun = None
                context_stack = context_stack[:-(len(stuff[1]))]
            else:
                raise ValueError("Unrecognized token '{}'".format(token))

    return tagged, heads

    

In [104]:
ohboy = "(S (S (NP (CD One) (NN image)) (VP (VBZ shows) (NP (NP (DT a) (NN girl)) (PP (PP (IN with) (NP (DT an) (NN elbow))) (VP (NN bent) (S (VP (VBG holding) (NP (DT a) (NN bottle)) (PP (IN to) (NP (PRP$ her) (NNS lips)))))))))) (, ,) (CC and) (S (NP (DT the) (JJ other) (NN image)) (VP (VBZ shows) (S (NP (DT a) (NN girl)) (VP (VBG holding) (NP (DT a) (JJ green) (NN bottle)) (PP (IN in) (NP (DT the) (JJ other) (NN arm))) (, ,) (ADJP (ADJP (JJ close) (PP (IN to))) (CC but) (RB not) (VBG touching) (NP (PRP$ her) (NN mouth))))))) (. .))"

In [107]:
tagged, heads = get_all_ambiguities(ohboy)

In [108]:
tagged

[('One', 'CD'),
 ('image', 'NN'),
 ('shows', 'VBZ'),
 ('a', 'DT'),
 ('girl', 'NN'),
 ('with', 'IN'),
 ('an', 'DT'),
 ('elbow', 'NN'),
 ('bent', 'NN'),
 ('holding', 'VBG'),
 ('a', 'DT'),
 ('bottle', 'NN'),
 ('to', 'IN'),
 ('her', 'PRP$'),
 ('lips', 'NNS'),
 (',', ','),
 ('and', 'CC'),
 ('the', 'DT'),
 ('other', 'JJ'),
 ('image', 'NN'),
 ('shows', 'VBZ'),
 ('a', 'DT'),
 ('girl', 'NN'),
 ('holding', 'VBG'),
 ('a', 'DT'),
 ('green', 'JJ'),
 ('bottle', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('other', 'JJ'),
 ('arm', 'NN'),
 (',', ','),
 ('close', 'JJ'),
 ('to', 'IN'),
 ('but', 'CC'),
 ('not', 'RB'),
 ('touching', 'VBG'),
 ('her', 'PRP$'),
 ('mouth', 'NN'),
 ('.', '.')]

In [109]:
heads

[(('image', 'NN'), 'NP'),
 (('girl', 'NN'), 'NP'),
 (('with', 'IN'), 'PP'),
 (('elbow', 'NN'), 'NP'),
 (('bottle', 'NN'), 'NP'),
 (('holding', 'VBG'), 'VP'),
 (('lips', 'NNS'), 'NP'),
 (('to', 'IN'), 'PP'),
 (('image', 'NN'), 'NP'),
 (('girl', 'NN'), 'NP'),
 (('bottle', 'NN'), 'NP'),
 (('in', 'IN'), 'PP'),
 (('arm', 'NN'), 'NP'),
 (('to', 'IN'), 'PP'),
 (('touching', 'VBG'), 'VP'),
 (('mouth', 'NN'), 'NP')]

In [110]:
ohboy

'(S (S (NP (CD One) (NN image)) (VP (VBZ shows) (NP (NP (DT a) (NN girl)) (PP (PP (IN with) (NP (DT an) (NN elbow))) (VP (NN bent) (S (VP (VBG holding) (NP (DT a) (NN bottle)) (PP (IN to) (NP (PRP$ her) (NNS lips)))))))))) (, ,) (CC and) (S (NP (DT the) (JJ other) (NN image)) (VP (VBZ shows) (S (NP (DT a) (NN girl)) (VP (VBG holding) (NP (DT a) (JJ green) (NN bottle)) (PP (IN in) (NP (DT the) (JJ other) (NN arm))) (, ,) (ADJP (ADJP (JJ close) (PP (IN to))) (CC but) (RB not) (VBG touching) (NP (PRP$ her) (NN mouth))))))) (. .))'